# Module 1: Building DevAssist - Project Analyzer

## Your AI Development Partner

In this course, we'll build **DevAssist** - an AI-powered development assistant that can analyze, modify, and test codebases. By the end of all three modules, you'll have a complete multi-agent development workflow.

### What We're Building Across All Modules

| Module | Component | Capability |
|--------|-----------|------------|
| **Module 1** | Project Analyzer | Understand any codebase |
| Module 2 | Feature Implementer | Make code changes with tracking |
| Module 3 | Full Dev Workflow | Multi-agent research, implement, test, document |

### This Module's Goals

Build an agent that can:
1. Scan a project and detect its tech stack
2. Map the project structure and find key files
3. Generate a comprehensive project summary

### The Sample Project

We'll work with **TaskFlow** - a simple Python CLI task manager located in `../sample_project/`. This is a realistic project you'll analyze, modify, and extend throughout the course.

---

## 1. Setup

In [ ]:
import os
from getpass import getpass

# Ensure API key is set
if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your Anthropic API key: ")

In [ ]:
from claude_agent_sdk import (
    query,
    ClaudeAgentOptions,
    AssistantMessage,
    ToolUseMessage,
    ToolResultMessage,
    ResultMessage
)

# Path to our sample project
PROJECT_PATH = os.path.abspath("../sample_project")
print(f"Sample project path: {PROJECT_PATH}")

---

## 2. Exercise 1: Basic Project Scanner

Let's start with the simplest possible agent - one that can look at a project and tell us what it is.

### The Minimal Agent

The Claude Agent SDK makes it easy to create an agent with just a few lines:

In [ ]:
async def basic_scanner(project_path: str):
    """The simplest project scanner - just asks Claude to look at the project."""
    
    prompt = f"""Look at the project in {project_path} and tell me:
    1. What programming language(s) it uses
    2. What the project does (in one sentence)
    3. The main entry point file"""
    
    async for message in query(prompt=prompt):
        if isinstance(message, AssistantMessage) and message.content:
            print(message.content)

In [ ]:
# Run the basic scanner
await basic_scanner(PROJECT_PATH)

### Adding Tool Configuration

The basic agent works, but we can make it more focused by explicitly configuring which tools it can use:

In [ ]:
async def configured_scanner(project_path: str):
    """Scanner with explicit tool configuration."""
    
    # Only give read-only tools - safe for exploration
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep"],
        permission_mode="bypassPermissions"  # Safe since these are read-only
    )
    
    prompt = f"""Analyze the project at {project_path}.
    
    Determine:
    1. Programming language and version (check pyproject.toml or similar)
    2. Project purpose (read README if available)
    3. Dependencies (if any)
    4. Main entry point"""
    
    async for message in query(prompt=prompt, options=options):
        if isinstance(message, AssistantMessage) and message.content:
            print(message.content)
        elif isinstance(message, ToolUseMessage):
            print(f"  [Using {message.tool_name}...]")

In [ ]:
# Run with explicit configuration
await configured_scanner(PROJECT_PATH)

### Key Insight: Tool Selection

| Tool | Purpose | When to Use |
|------|---------|-------------|
| `Glob` | Find files by pattern | "Find all Python files" |
| `Grep` | Search file contents | "Find where X is used" |
| `Read` | Read file contents | "Show me this file" |

For a read-only scanner, these three tools are all we need!

---

## 3. Exercise 2: Structure Mapper with Streaming

Now let's build a more sophisticated scanner that maps the entire project structure and shows us its progress in real-time.

### Understanding Streaming

Streaming lets us see what the agent is doing as it works:

In [ ]:
async def structure_mapper(project_path: str):
    """Map project structure with real-time progress."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep"],
        permission_mode="bypassPermissions"
    )
    
    prompt = f"""Analyze the structure of the project at {project_path}.
    
    Create a map that includes:
    1. All Python files and their purpose
    2. Test files and what they test
    3. Configuration files
    4. Key classes and functions in each file
    
    Present as a structured overview."""
    
    tool_count = 0
    
    print("Mapping project structure...\n")
    
    async for message in query(prompt=prompt, options=options):
        if isinstance(message, ToolUseMessage):
            tool_count += 1
            print(f"[Step {tool_count}] {message.tool_name}: {str(message.tool_input)[:60]}...")
            
        elif isinstance(message, ToolResultMessage):
            status = "done" if message.success else "failed"
            print(f"         -> {status}")
            
        elif isinstance(message, AssistantMessage) and message.content:
            print(f"\n{'='*60}")
            print("PROJECT STRUCTURE MAP")
            print(f"{'='*60}\n")
            print(message.content)

In [ ]:
# Map the project structure
await structure_mapper(PROJECT_PATH)

### Message Types Explained

| Message Type | When It Appears | What It Contains |
|--------------|-----------------|------------------|
| `AssistantMessage` | Claude speaks | Text response or reasoning |
| `ToolUseMessage` | Before tool runs | Tool name and input |
| `ToolResultMessage` | After tool runs | Success status and output |
| `ResultMessage` | Session ends | Session ID for resuming |

---

## 4. Exercise 3: Comprehensive Project Analyzer

Now let's build the full DevAssist Project Analyzer that generates a complete project summary.

### The DevAssist Analyzer

In [ ]:
async def devassist_analyzer(project_path: str) -> dict:
    """Complete project analyzer - the foundation of DevAssist."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep"],
        permission_mode="bypassPermissions"
    )
    
    prompt = f"""You are DevAssist, an AI development partner. Perform a comprehensive analysis 
    of the project at {project_path}.
    
    Generate a complete project report covering:
    
    ## 1. Project Overview
    - Name and description
    - Programming language and version
    - Project type (CLI, web app, library, etc.)
    
    ## 2. Architecture
    - Main modules and their responsibilities
    - Key classes and functions
    - Data flow (how data moves through the system)
    
    ## 3. Entry Points
    - Main entry point(s)
    - Available commands or APIs
    - Configuration options
    
    ## 4. Dependencies
    - External dependencies
    - Internal module dependencies
    
    ## 5. Testing
    - Test files and coverage areas
    - How to run tests
    
    ## 6. Potential Improvements
    - Missing features (based on TODOs or gaps)
    - Code quality observations
    - Suggested enhancements
    
    Be thorough but concise."""
    
    result = {
        "tools_used": [],
        "analysis": ""
    }
    
    print("DevAssist is analyzing the project...\n")
    
    async for message in query(prompt=prompt, options=options):
        if isinstance(message, ToolUseMessage):
            result["tools_used"].append(message.tool_name)
            print(f"  Examining: {message.tool_name}...")
            
        elif isinstance(message, AssistantMessage) and message.content:
            result["analysis"] = message.content
            
        elif isinstance(message, ResultMessage):
            result["session_id"] = message.session_id
    
    print(f"\nAnalysis complete! Used {len(result['tools_used'])} tool calls.")
    return result

In [ ]:
# Run the full analyzer
analysis = await devassist_analyzer(PROJECT_PATH)

print("\n" + "="*60)
print("DEVASSIST PROJECT ANALYSIS")
print("="*60 + "\n")
print(analysis["analysis"])

### Saving the Session

Notice we captured the `session_id`. This lets us continue the conversation later!

In [ ]:
# Save session for later use
print(f"Session ID: {analysis.get('session_id', 'N/A')[:20]}...")
print("\nYou can resume this session in Module 2 to implement features!")

---

## 5. Single Query vs Streaming Comparison

Let's compare both modes to understand when to use each:

In [ ]:
import time

async def compare_modes(project_path: str):
    """Compare streaming vs collecting all results."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Glob", "Read"],
        permission_mode="bypassPermissions"
    )
    
    prompt = f"List all Python files in {project_path} and count them."
    
    # Mode 1: Collect all (like single query)
    print("=== Mode 1: Collect All ===")
    start = time.time()
    messages = []
    async for msg in query(prompt=prompt, options=options):
        messages.append(msg)
    elapsed = time.time() - start
    print(f"Time: {elapsed:.2f}s")
    print(f"Messages collected: {len(messages)}")
    print(f"Final response available after: {elapsed:.2f}s\n")
    
    # Mode 2: Stream with progress
    print("=== Mode 2: Stream with Progress ===")
    start = time.time()
    async for msg in query(prompt=prompt, options=options):
        elapsed = time.time() - start
        msg_type = type(msg).__name__
        print(f"[{elapsed:.2f}s] {msg_type}")
    print(f"\nTotal time: {time.time() - start:.2f}s")

In [ ]:
await compare_modes(PROJECT_PATH)

### When to Use Each Mode

| Mode | Use When |
|------|----------|
| **Collect All** | Background processing, batch jobs, when you need the final result only |
| **Streaming** | User-facing apps, long tasks, when progress feedback matters |

---

## 6. Try It Yourself: Analyze a Different Aspect

Create your own analyzer that focuses on a specific aspect of the project:

In [ ]:
async def security_scanner(project_path: str):
    """Scan for potential security issues."""
    
    options = ClaudeAgentOptions(
        allowed_tools=["Read", "Glob", "Grep"],
        permission_mode="bypassPermissions"
    )
    
    prompt = f"""Analyze the project at {project_path} for potential security issues.
    
    Check for:
    - Hardcoded credentials or API keys
    - SQL injection vulnerabilities
    - Unsafe file operations
    - Input validation issues
    
    Report any findings with file locations and severity."""
    
    async for message in query(prompt=prompt, options=options):
        if isinstance(message, AssistantMessage) and message.content:
            print(message.content)
        elif isinstance(message, ToolUseMessage):
            print(f"  [Scanning with {message.tool_name}...]")

In [ ]:
# Run the security scanner
await security_scanner(PROJECT_PATH)

---

## Summary

In this module, you built the first component of DevAssist - the **Project Analyzer**.

### What You Learned

| Concept | Implementation |
|---------|----------------|
| Basic agent | `query(prompt="...")` |
| Tool configuration | `ClaudeAgentOptions(allowed_tools=[...])` |
| Permission modes | `permission_mode="bypassPermissions"` for read-only |
| Streaming | Process messages as they arrive |
| Message types | AssistantMessage, ToolUseMessage, ResultMessage |
| Session capture | Save `session_id` for later |

### The DevAssist Analyzer Can:

1. Detect tech stack and language
2. Map project structure
3. Identify key files and their purposes
4. Find dependencies and entry points
5. Suggest improvements

### Coming in Module 2

We'll extend DevAssist to actually **implement features**:
- Use `Edit` tool to modify code
- Track progress with `TodoWrite`
- Maintain context with sessions
- Create a custom "implement-feature" skill

### Quick Reference

```python
# Basic read-only agent
options = ClaudeAgentOptions(
    allowed_tools=["Read", "Glob", "Grep"],
    permission_mode="bypassPermissions"
)

async for msg in query(prompt="...", options=options):
    if isinstance(msg, AssistantMessage):
        print(msg.content)  # Claude's response
    elif isinstance(msg, ToolUseMessage):
        print(f"Using {msg.tool_name}")  # Tool being used
    elif isinstance(msg, ResultMessage):
        session_id = msg.session_id  # Save for later
```